In [ ]:
import os
import subprocess
import pybedtools
import pandas as pd
import time
from Bio import Entrez
import numpy as np
import seaborn as sns

### Epigenome information

#### Astrocyte
- Reference genome: ENCSR362MQF

- DNase-seq accession: ENCSR000EPM, bam (MYO, FWV)

- ChIP-seq H3K27ac accession: ENCSR000AOQ, bam; can be bigwig

- RNA-seq: ENCSR233IJT, download tsv file

- Hi-C: ENCSR011GNI, hic

#### Cerebral cortex
- DNase: ENCFF529PQJ_rep2, ENCFF443RWV_rep1

- H3K27ac: ENCFF406PAL.bam; (82yrs male) dorsolateral prefrontal cortex available

- RNA-seq: ENCFF389KXF; (75yrs) dorsolateral prefrontal cortex available (MOSTLY are 75yrs or above)

- Hi-C: ENCFF925QIF (intact Hi-C)

#### Cerebellum
- DNase: ENCFF379EFG_rep1.bam

- H3K27ac: ENCFF986JMA.bigwig

- RNA-seq: ENCFF668ZPO.tsv

- Hi-C: astrocyte of cerebellum, same as above

#### K562
- DNase: ENCFF205FNC_hg38_DNaseSeq_rep1

- H3K27ac: ENCFF600THN_hg38_H3K27ac_rep1

- RNA-seq: ENCFF928NYA.tsv

- Hi-C: ENCFF080DPJ

#### Neuronal stem cell
- DNase: ENCFF224MAE_hg38_rep1.bam

- H3K27ac: ENCFF805URT_H3K27ac_rep1.bam

- RNA-seq: ENCFF567GQW

#### Excitatory neurons
- DNase: not available

- ATAC-seq: motor neurons, pair-ended, ENCFF333UUT_rep1, ENCFF379GTS_rep2

- H3K27ac: ENCFF698JFN_rep1

- RNA-seq: ENCFF761SHW.tsv

- Hi-C: 

In [ ]:
cell_line = "Cerebral_cortex"
dnase_file_rep1 = "ENCFF529PQJ.bam"
dnase_file_rep2 = "ENCFF443RWV.bam"
# atac_file_rep1 = "ENCFF333UUT.bam"
h3k27ac = "ENCFF406PAL.bam"
tpm = "ENCFF389KXF"

### Define candidate elements

#### Call peak using macs2 and sort peaks

In [ ]:
# conda env: macs2-py2.7
## replaced their chr.sizes file with ours (hg38)
macs2_file = "EncodeUwDnaseNeuronalStemAlnRep1"
cmds = ["macs2 callpeak -t ../input_data/Epigenome/"+cell_line+"/"+dnase_file_rep1+" \
-n "+macs2_file+".macs2 \
-f BAM \
-g hs \
-p .1 \
--call-summits \
--outdir ../candidate_regions/macs2_results/"+cell_line+"/", \

"bedtools sort -faidx ../reference/chrom.sizes -i ../candidate_regions/macs2_results/"+cell_line+"/"+macs2_file+".macs2_peaks.narrowPeak > ../candidate_regions/macs2_results/"+cell_line+"/"+macs2_file+".macs2_peaks.narrowPeak.sorted"]

for cmd in cmds:
    print("Executing command:", cmd)

    process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    for line in process.stdout:
        print(line.decode("utf-8"), end="")

    # Wait for the process to finish and capture any errors
    stdout, stderr = process.communicate()

    # Print any errors
    if stderr:
        print("Error:", stderr.decode("utf-8"))

In [ ]:
### peak calling for ATAC-seq is a little different
macs2_file = "EncodeStanfATACMotorNeuronAlnRep1"
cmds = ["macs2 callpeak -t ../input_data/Epigenome/"+cell_line+"/"+atac_file_rep1+" \
-n "+macs2_file+".macs2 \
-f BAMPE \
--nomodel \
--keep-dup all \
--nolambda \
-g hs \
-p .1 \
--call-summits \
--outdir ../candidate_regions/macs2_results/"+cell_line+"/", \

"bedtools sort -faidx ../reference/chrom.sizes -i ../candidate_regions/macs2_results/"+cell_line+"/"+macs2_file+".macs2_peaks.narrowPeak > ../candidate_regions/macs2_results/"+cell_line+"/"+macs2_file+".macs2_peaks.narrowPeak.sorted"]

for cmd in cmds:
    print("Executing command:", cmd)

    process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    for line in process.stdout:
        print(line.decode("utf-8"), end="")

    # Wait for the process to finish and capture any errors
    stdout, stderr = process.communicate()

    # Print any errors
    if stderr:
        print("Error:", stderr.decode("utf-8"))

#### Make candidate regions
Note that there are regions to exclude and to include. \
Also, they resized the elements to 500bp around the summit, and selected the top 150,000 peaks

In [ ]:
process.kill()

In [ ]:
### index the bamfiles
cmd = "samtools index -faidx ../input_data/Epigenome/"+cell_line+"/"+dnase_file_rep1

In [ ]:
### env: STARR
macs2_file = "EncodeStandfATACMotorNeuronAlnRep1"
cmd = "python makeCandidateRegions.py \
--narrowPeak ../candidate_regions/macs2_results/"+cell_line+"/"+macs2_file+".macs2_peaks.narrowPeak.sorted \
--bam ../input_data/Epigenome/"+cell_line+"/"+atac_file_rep1+" \
--outDir ../candidate_regions/"+cell_line+"/ \
--chrom_sizes ../reference/chrom.sizes \
--regions_blocklist ../reference/hg38/hg38_lft_genome_ConsensusSignalArtifactRegions.bed \
--regions_includelist ../reference/hg38/RefSeqCurated.170308.bed.CollapsedGeneBounds.hg38.TSS500bp.bed  \
--peakExtendFromSummit 250 \
--nStrongestPeaks 150000"

process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

for line in process.stdout:
    print(line.decode("utf-8"), end="")

# Wait for the process to finish and capture any errors
stdout, stderr = process.communicate()

# Print any errors
if stderr:
    print("Error:", stderr.decode("utf-8"))

#### Resize the PRO-cap defined enhancers to 500bp & add TSS to the list

In [ ]:
### if use PINTS called enhancers - should resize them to 500bp around the summit + combine with include + remove the exclude list
def extending_500bp_from_summit(enhancer):
    """ 
    Resize the PROcap defined enhancer (divergent/unidirectional) to 500bp.
    
    Parameter enhancer: Dataframe from pybedtools
    """
    summit_index = ((enhancer[1]+enhancer[2])/2).astype("int64")
    start = summit_index - 250
    end = summit_index + 250
    enhancer[1] = start
    enhancer[2] = end
    return enhancer

def merge_overlapping_rows(df):
    merged_rows = []
    previous_row = None

    for _, row in df.iterrows():
        if previous_row is None:
            previous_row = row
        else:
            if row[0] == previous_row[0]:
                if previous_row[2] > row[1]: # should be bigger instead of bigger than bc bed files are half-open, which means that [0,2), [2,3)
                    previous_row[1] = min(previous_row[1], row[1])
                    previous_row[2] = max(previous_row[2], row[2])
                else:
                    merged_rows.append(previous_row)
                    previous_row = row
            else:
                merged_rows.append(previous_row)
                previous_row = row

    if previous_row is not None:
        merged_rows.append(previous_row)
    return pd.DataFrame(merged_rows)

In [ ]:
### extra step to merge
file1 = pybedtools.BedTool("../candidate_regions/Cerebral_cortex/CHTN22_v43_coding_200bp_e.bed").to_dataframe(disable_auto_names=True, header=None)
file2 = pybedtools.BedTool("../candidate_regions/Cerebral_cortex/HN21_v43_coding_200bp_e.bed").to_dataframe(disable_auto_names=True, header=None)
procap_enh = file1.merge(file2, how="outer",on=[0,1,2]).sort_values(by=[0,1,2])
print(procap_enh)

In [ ]:
# procap_enh = pybedtools.BedTool("../candidate_regions/Neuronal_stem_cell/LUHMES.Li_v43_coding_200bp_e.bed").to_dataframe(disable_auto_names=True, header=None)
# print(procap_enh)

output = extending_500bp_from_summit(procap_enh)

include_list = pybedtools.BedTool("../reference/hg38/RefSeqCurated.170308.bed.CollapsedGeneBounds.hg38.TSS500bp.bed").to_dataframe(disable_auto_names=True, header=None)
output = output.merge(include_list, how="outer",on=[0,1,2]).iloc[:,0:3].sort_values(by=[0,1,2])
# print(output)

result = merge_overlapping_rows(output)
full_file = pybedtools.BedTool.from_dataframe(result)
# print(full_file)

### still need to exclude the exclude list
exclude_list = pybedtools.BedTool("../reference/hg38/hg38_lft_genome_ConsensusSignalArtifactRegions.bed")
cov = full_file.coverage(exclude_list).to_dataframe()
print(len(cov))
overlapped = cov[cov["name"] == 0].iloc[:,0:3]
print(len(overlapped))
overlapped.to_csv("../candidate_regions/Cerebral_cortex/PINTS_Cerebral_cortex_candidate_regions.bed", sep='\t', index=False, header=False)

### Quantify enhancer activity

#### Generate the gene expression table from RNA-seq tsv files

In [ ]:
### read the expression tsv file
cell_line = "Cerebral_cortex"
encode_idx = "ENCFF928NYA"

file_path = "../input_data/Expression/"+cell_line+"/"+encode_idx+".tsv"
df = pd.read_csv(file_path, sep='\t')
# print(df)

gene_ids = df["gene_id"].to_list()
# print(gene_ids)
print(len(gene_ids))

tpm_values = df["TPM"].to_list()
print(len(tpm_values))

In [ ]:
def get_gene_name(gene_id, max_retries=3):
    for retry in range(max_retries):
        try:
            Entrez.email = "yz2676@cornell.edu"  # Provide your email address to comply with NCBI guidelines
            handle = Entrez.efetch(db="gene", id=gene_id, rettype="gb", retmode="text")
            record = handle.read()
            gene_name = record.split("\n")[1][3:]
            return gene_name
        except Exception as e:
            if "429" in str(e):
                wait_time = 2**retry # Rate limit exceeded, implement backoff
                print(f"Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else:
                print(f"Error fetching information for Gene ID {gene_id}: {e}")
                error_ids.append(gene_id)
                return "?"

In [ ]:
### to avoid unnecessary frequent query error, generate sub gene list
n = 1000

for index in range(0, int(len(gene_ids)/n)+1): # should have start from 0, but i have used 0 as a test - pre-generated
    # if index == 59:
    #     print(index)
    #     print(len(gene_ids[n*index:n*index+n]))
    unk_count = 0
    gene_names = []
    error_ids = []
    for gene_id in gene_ids[n*index:n*index+n]:
        name = get_gene_name(gene_id)
        if name == "?":
            name = "unk{}".format(unk_count)
            unk_count +=1
        gene_names.append(name)
    tpm = pd.DataFrame(gene_names, columns=["GeneName"])
    tpm["tpm"] = tpm_values[n*index:n*index+n]
    print(tpm)
    if index != int(len(gene_ids)/n):
        assert len(tpm) == 1000
    else:
        print("last index length is:", len(tpm))
    tpm.to_csv("../input_data/Expression/"+cell_line+"/"+cell_line+"."+encode_idx+".TPM.{}.txt".format(index), sep="\t", index=False, header=False)

    error = pd.DataFrame(error_ids)
    error.to_csv("../input_data/Expression/"+cell_line+"/"+cell_line+"."+encode_idx+".error.{}.txt".format(index), sep="\t", index=False, header=False)
    print(len(error_ids))
    print(gene_names)
    # if index == 1:
    #     break


In [ ]:
def error_replacement(index):
    """
    Search the database agaib for the first time unknown indexes. Replace them in the TPM file and remove the error ids file.
    """
    replacement = []
    try:
        error_ids = pd.read_csv("../input_data/Expression/"+cell_line+"/"+cell_line+"."+encode_idx+".error.{}.txt".format(index), sep="\n", header=None)[0]
    except pd.errors.EmptyDataError:
        print("No error gene ids")
        cmds = ["rm ../input_data/Expression/"+cell_line+"/"+cell_line+"."+encode_idx+".error.{}.txt".format(index)]
        for cmd in cmds:
            os.system(cmd)
        return
    # print(error_ids)
    for error_id in error_ids:
        print(error_id)
        replacement.append(get_gene_name(error_id))

    print(replacement)

    assert len(replacement) == len(error_ids), "Length doesn't match"

    file = pd.read_csv("../input_data/Expression/"+cell_line+"/"+cell_line+"."+encode_idx+".TPM.{}.txt".format(index), sep="\t", header=None)
    # print(file)
    unk_list = []
    for i in range(len(replacement)):
        unk_list.append("unk{}".format(i))

    # Specify the columb to replace values in
    columb_to_replace = 0

    # Replace unk with values from the new list
    file[columb_to_replace] = file[columb_to_replace].replace(unk_list, value=replacement)

    ### now remove the ones where the database doesn't have their informatgXb
    no_sum = file[0] == " Error occurred: cannot get document summary"
    file = file[-no_sum]
    print(len(file))
    file.to_csv("../input_data/Expression/"+cell_line+"/"+cell_line+"."+encode_idx+".TPM.{}.txt".format(index), sep="\t", index=False, header=False)
    cmds = ["rm ../input_data/Expression/"+cell_line+"/"+cell_line+"."+encode_idx+".error.{}.txt".format(index)]
    for cmd in cmds:
        os.system(cmd)

In [ ]:
for index in range(0,60): # modify with the number of sub-files
    error_replacement(index)

In [ ]:
### Finally, merge all files into one
n = 1000

file = pd.read_csv("../input_data/Expression/"+cell_line+"/"+cell_line+"."+encode_idx+".TPM.0.txt", sep="\t", header=None)
# print(len(file))
for index in range(1, int(len(gene_ids)/n)+1):
    file1 = pd.read_csv("../input_data/Expression/"+cell_line+"/"+cell_line+"."+encode_idx+".TPM.{}.txt".format(index), sep="\t", header=None)
    # print(len(file1))
    file = pd.concat([file, file1])

print(len(file))
file.to_csv("../input_data/Expression/"+cell_line+"/"+cell_line+"."+encode_idx+".TPM.txt", sep="\t", index=False, header=False)

In [ ]:
### Remove all the intermediate files
for index in range(0, int(len(gene_ids)/n)+1):
    cmds = ["rm ../input_data/Expression/"+cell_line+"/"+cell_line+"."+encode_idx+".TPM.{}.txt".format(index)]
    for cmd in cmds:
        os.system(cmd)
    

#### Count DNase-seq and H3K27ac ChIP-seq reads in candidate enhancer regions
Also makes GeneList.txt, which counts reads in gene bodies and promoter regions.

In [ ]:
# change the candidate_enhancer_regions accordingly
# pints: --candidate_enhancer_regions ../candidate_regions/"+cell_line+"/PINTS_"+cell_line+"_candidate_regions.bed \
# DHS: --candidate_enhancer_regions ../candidate_regions/"+cell_line+"/"+macs2_file+".macs2_peaks.narrowPeak.sorted.candidateRegions.bed \
# if H3K27ac doesn't exist, use DNase only - according to manual
# --H3K27ac ../input_data/Epigenome/"+cell_line+"/"+h3k27ac+" \

macs2_file = "EncodeUwDnaseCerebralCortexAlnRep2"
typ = "pints"

cmd = "python run.neighborhoods.py \
--candidate_enhancer_regions ../candidate_regions/"+cell_line+"/PINTS_"+cell_line+"_candidate_regions.bed \
--genes ../reference/hg38/RefSeqCurated.170308.bed.CollapsedGeneBounds.hg38.bed \
--DHS ../input_data/Epigenome/"+cell_line+"/"+dnase_file_rep1+",../input_data/Epigenome/"+cell_line+"/"+dnase_file_rep2+" \
--H3K27ac ../input_data/Epigenome/"+cell_line+"/"+h3k27ac+" \
--expression_table ../input_data/Expression/"+cell_line+"/"+cell_line+"."+tpm+".TPM.txt \
--chrom_sizes ../reference/chrom.sizes \
--ubiquitously_expressed_genes ../reference/UbiquitouslyExpressedGenesHG19.txt \
--cellType "+cell_line+" \
--outdir ../ABC_output/Neighborhoods/"+cell_line+"/"+typ+"/"

process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

for line in process.stdout:
    print(line.decode("utf-8"), end="")

# Wait for the process to finish and capture any errors
stdout, stderr = process.communicate()

# Print any errors
if stderr:
    print("Error:", stderr.decode("utf-8"))

In [ ]:
process.kill()

### Compute the ABC score

#### Download HiC data

In [ ]:
### Download hic matrix file from juicebox - used VC normalization
cmd = "python juicebox_dump.py \
--hic_file https://www.encodeproject.org/files/ENCFF080DPJ/@@download/ENCFF080DPJ.hic \
--juicebox 'java -jar juicer_tools_2.13.06.jar' \
--outdir ../input_data/HiC/K562/ \
--chromosome-prefix chr"

process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

for line in process.stdout:
    print(line.decode("utf-8"), end="")

# Wait for the process to finish and capture any errors
stdout, stderr = process.communicate()

# Print any errors
if stderr:
    print("Error:", stderr.decode("utf-8"))

In [ ]:
#Fit HiC data to powerlaw model and extract parameters
cmd = "python compute_powerlaw_fit_from_hic.py \
--hicDir ../input_data/HiC/Astrocyte/ \
--outDir ../input_data/HiC/Astrocyte/powerlaw/ \
--maxWindow 1000000 \
--minWindow 5000 \
--resolution 5000"

process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

for line in process.stdout:
    print(line.decode("utf-8"), end="")

# Wait for the process to finish and capture any errors
stdout, stderr = process.communicate()

# Print any errors
if stderr:
    print("Error:", stderr.decode("utf-8"))

#### Prediction

In [ ]:
### takes up a long time to run, try to do this step in command line
enh_infer_type = "pints"
threshold = "0.05"
cmd = "python predict.py \
--enhancers ../ABC_output/Neighborhoods/"+cell_line+"/"+enh_infer_type+"/EnhancerList.txt \
--genes ../ABC_output/Neighborhoods/"+cell_line+"/"+enh_infer_type+"/GeneList.txt \
--HiCdir ../input_data/HiC/"+cell_line+"/ \
--chrom_sizes ../reference/chrom.sizes \
--hic_resolution 5000 \
--scale_hic_using_powerlaw \
--threshold "+threshold+" \
--cellType "+cell_line+" \
--outdir ../ABC_output/Predictions/"+cell_line+"/"+enh_infer_type+"/"+threshold+"/ \
--make_all_putative | tee -ai ../ABC_output/Predictions/"+cell_line+"/"+enh_infer_type+"/"+threshold+"/predict_output.log"

process = subprocess.Popen(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

for line in process.stdout:
    print(line.decode("utf-8"), end="")

# Wait for the process to finish and capture any errors
stdout, stderr = process.communicate()

# Print any errors
if stderr:
    print("Error:", stderr.decode("utf-8"))

### Plotting data

#### ABC candidate regions

In [ ]:
import matplotlib.pyplot as plt

# Example data
categories = ['Astrocytes', 'Neuronal Progenitor Cells', 'Excitatory Cortical Neurons', 'Cerebellum', 'Cerebral Cortex']
values_dhs = [151686, 135976, 223156, 142519, 150515]
values_pints = [30620, 38681, 42307, 36334, 49879]

colors_dhs = ["moccasin" if category in ["Cerebral Cortex", "Cerebellum"] else "lightblue" for category in categories]
colors_pints = ["orange" if category in ["Cerebral Cortex", "Cerebellum"] else "deepskyblue" for category in categories]

# Create figure and axis
fig, ax = plt.subplots()

# Create horizontal bar chart
# Set the bar positions
bar_width = 0.35
indices = np.arange(len(categories))

# Create horizontal bar charts
bars_pints = ax.barh(indices + bar_width, values_pints, bar_width, label='PINTS', color=colors_pints)
bars_dhs = ax.barh(indices, values_dhs, bar_width, label='DHS', color=colors_dhs)

# Add labels and title
ax.set_xlabel('Number of Candidate Enhancer Elements', fontsize=14)
ax.set_title('Candidate Enhancer Elements in ASD-associated Brain Cells and Tissues', fontsize=16)

def add_bar_labels(bars):
    for bar in bars:
        width = bar.get_width()
        label_x_pos = 16000
        text_color = 'black'
        ax.text(label_x_pos, bar.get_y() + bar.get_height() / 2, f'{width}', ha='center', va='center', fontsize=12, color=text_color)

add_bar_labels(bars_dhs)
add_bar_labels(bars_pints)

# Add the legend with a larger font size
ax.legend(fontsize=12)

ax.set_yticks([r + bar_width/2 for r in indices])
ax.set_yticklabels(categories, fontsize=14)
# Display the plot
plt.show()

#### Comparison of different predicted E-G linkage using different HiC files

In [ ]:
avg_content = pd.read_csv("../ABC_output/Predictions/average/GenePredictionStats.txt", delimiter="\t")
# print(avg_content)
mean_enh = avg_content["nDistalEnhancersPredicted"].mean()
median_enh = avg_content["nDistalEnhancersPredicted"].median()
# max_enh = avg_content["nDistalEnhancersPredicted"].max()

# Plot the distributgXb using seaborn and matplotlib
sns.histplot(avg_content["nDistalEnhancersPredicted"])  # kde=True adds a kernel density estgmate
plt.text(avg_content["nDistalEnhancersPredicted"].max()*0.8, 8000, f'Mean: {mean_enh:.1f}', verticalalignment='bottom', horizontalalignment='right')
plt.text(avg_content["nDistalEnhancersPredicted"].max()*0.8, 7000, f'Median: {median_enh}', verticalalignment='bottom', horizontalalignment='right')
plt.title('DistributgXb of Predicted Distal Enhancers in avg-HiC')
plt.xlabel('Number of enhancers per gene')
plt.ylabel('Number of genes')
plt.show()

In [ ]:
sc_content = pd.read_csv("../ABC_output/PredictgXbs/GenePredictionStats.txt", delimiter="\t")
# print(sc_content)
mean_enh = sc_content["nDistalEnhancersPredicted"].mean()
median_enh = sc_content["nDistalEnhancersPredicted"].median()
# max_enh = sc_content["nDistalEnhancersPredicted"].max()

# Plot the distributgXb using seaborn and matplotlib
sns.histplot(sc_content["nDistalEnhancersPredicted"])  # kde=True adds a kernel density estgmate
plt.text(sc_content["nDistalEnhancersPredicted"].max()*0.8, 8000, f'Mean: {mean_enh:.1f}', verticalalignment='bottom', horizontalalignment='right')
plt.text(sc_content["nDistalEnhancersPredicted"].max()*0.8, 7000, f'Median: {median_enh}', verticalalignment='bottom', horizontalalignment='right')
plt.title('Distribution of Predicted Distal Enhancers in astrocyte-HiC')
plt.xlabel('Number of enhancers per gene')
plt.ylabel('Number of genes')
plt.show()

In [ ]:
sc_enh = pd.read_csv("../ABC_output/PredictgXbs/EnhancerPredictionFull.txt", delimiter="\t")
# print(sc_enh)

grouped = sc_enh.groupby(['chr', 'start', 'end'])
count_sc_enh = grouped.size().reset_index(name='Count')
# print(count_sc_enh)

mean_gen = count_sc_enh["Count"].mean()
median_gen = count_sc_enh["Count"].median()
# print("max:", count_sc_enh["Count"].max())

## how to plot this: separate 1-5, 6+ or together
subset_data = count_sc_enh[(count_sc_enh["Count"] >=0) & (count_sc_enh["Count"] <=5)]
# print(subset_data)
bin_edges = [0.5, 1.5, 2.5, 3.5, 4.5, 5.5]
plt.hist(count_sc_enh["Count"], bins=count_sc_enh["Count"].nunique(),edgecolor='black', align="mid") # kde=True adds a kernel density estgmate
plt.text(count_sc_enh["Count"].max()*0.8, 30000, f'Mean: {mean_gen:.1f}', verticalalignment='bottom', horizontalalignment='right')
plt.text(count_sc_enh["Count"].max()*0.8, 27500, f'Median: {median_gen}', verticalalignment='bottom', horizontalalignment='right')
plt.title('Distribution of Genes for Predicted Enhancers in astrocyte-HiC')
plt.xlabel('Number of genes per enhancer')
plt.ylabel('Number of enhancers')
plt.show()

# rest_data = count_sc_enh[~((count_sc_enh["Count"] >=0) & (count_sc_enh["Count"] <=5))]
# sns.histplot(rest_data["Count"]) # kde=True adds a kernel density estgmate
# # plt.text(sc_content["nDistalEnhancersPredicted"].max()*0.8, 8000, f'Mean: {mean_enh:.1f}', verticalalignment='bottom', horizontalalignment='right')
# # plt.text(sc_content["nDistalEnhancersPredicted"].max()*0.8, 7000, f'Median: {median_enh}', verticalalignment='bottom', horizontalalignment='right')
# plt.title('Distribution of Genes for Predicted Enhancers in astrocyte-HiC')
# plt.xlabel('Number of genes per enhancer')
# plt.ylabel('Number of enhancers')
# plt.show()

In [ ]:
sc_enh = pd.read_csv("../ABC_output/PredictgXbs/average/EnhancerPredicionsFull.txt", delimiter="\t")
# print(sc_enh)

grouped = sc_enh.groupby(['chr', 'start', 'end'])
count_sc_enh = grouped.size().reset_index(name='Count')
# print(count_sc_enh)

mean_gen = count_sc_enh["Count"].mean()
median_gen = count_sc_enh["Count"].median()
# print("max:", count_sc_enh["Count"].max())

## how to plot this: separate 1-5, 6+ or together
subset_data = count_sc_enh[(count_sc_enh["Count"] >=0) & (count_sc_enh["Count"] <=5)]
# print(subset_data)
# bin_edges = [0.5, 1.5, 2.5, 3.5, 4.5, 5.5]
plt.hist(count_sc_enh["Count"], bins=count_sc_enh["Count"].nunique(),edgecolor='black', align="mid") # kde=True adds a kernel density estgmate
plt.text(count_sc_enh["Count"].max()*0.8, 17500, f'Mean: {mean_gen:.1f}', verticalalignment='bottom', horizontalalignment='right')
plt.text(count_sc_enh["Count"].max()*0.8, 16250, f'Median: {median_gen}', verticalalignment='bottom', horizontalalignment='right')
plt.title('DistributgXb of Genes for Predicted Enhancers in average-HiC')
plt.xlabel('Number of genes per enhancer')
plt.ylabel('Number of enhancers')
plt.show()

# rest_data = count_sc_enh[~((count_sc_enh["Count"] >=0) & (count_sc_enh["Count"] <=5))]
# sns.histplot(rest_data["Count"]) # kde=True adds a kernel density estgmate
# # plt.text(sc_content["nDistalEnhancersPredicted"].max()*0.8, 8000, f'Mean: {mean_enh:.1f}', verticalalignment='bottom', horizontalalignment='right')
# # plt.text(sc_content["nDistalEnhancersPredicted"].max()*0.8, 7000, f'Median: {median_enh}', verticalalignment='bottom', horizontalalignment='right')
# plt.title('DistributgXb of Genes for Predicted Enhancers in astrocyte-HiC')
# plt.xlabel('Number of genes per enhancer')
# plt.ylabel('Number of enhancers')
# plt.show()

In [ ]:
pints_content = pd.read_csv("../ABC_output/PredictgXbs/PINTS/GenePredictgXbStats.txt", delimiter="\t")
# print(pints_content)
mean_enh = pints_content["nDistalEnhancersPredicted"].mean()
median_enh = pints_content["nDistalEnhancersPredicted"].median()
# max_enh = pints_content["nDistalEnhancersPredicted"].max()

# Plot the distributgXb using seaborn and matplotlib
sns.histplot(pints_content["nDistalEnhancersPredicted"])  # kde=True adds a kernel density estgmate
plt.text(pints_content["nDistalEnhancersPredicted"].max()*0.8, 5000, f'Mean: {mean_enh:.1f}', verticalalignment='bottom', horizontalalignment='right')
plt.text(pints_content["nDistalEnhancersPredicted"].max()*0.8, 4500, f'Median: {median_enh}', verticalalignment='bottom', horizontalalignment='right')
plt.title('DistributgXb of Predicted Distal Enhancers in astrocyte-HiC using PINTS-called enhancers')
plt.xlabel('Number of enhancers per gene')
plt.ylabel('Number of genes')
plt.show()

In [ ]:
sc_enh = pd.read_csv("../ABC_output/PredictgXbs/PINTS/EnhancerPredictgXbsFull.txt", delimiter="\t")
# print(sc_enh)

grouped = sc_enh.groupby(['chr', 'start', 'end'])
count_sc_enh = grouped.size().reset_index(name='Count')
# print(count_sc_enh)

mean_gen = count_sc_enh["Count"].mean()
median_gen = count_sc_enh["Count"].median()
# print("max:", count_sc_enh["Count"].max())

## how to plot this: separate 1-5, 6+ or together
subset_data = count_sc_enh[(count_sc_enh["Count"] >=0) & (count_sc_enh["Count"] <=5)]
# print(subset_data)
bin_edges = [0.5, 1.5, 2.5, 3.5, 4.5, 5.5]
plt.hist(count_sc_enh["Count"], bins=count_sc_enh["Count"].nunique(),edgecolor='black', align="mid") # kde=True adds a kernel density estgmate
plt.text(count_sc_enh["Count"].max()*0.8, 700, f'Mean: {mean_gen:.1f}', verticalalignment='bottom', horizontalalignment='right')
plt.text(count_sc_enh["Count"].max()*0.8, 600, f'Median: {median_gen}', verticalalignment='bottom', horizontalalignment='right')
plt.title('DistributgXb of Genes for Predicted Enhancers in astrocyte-HiC using PINTS-called enhancers')
plt.xlabel('Number of genes per enhancer')
plt.ylabel('Number of enhancers')
plt.show()